## Using a pre-trained model with transfer learning: InceptionV3

### Install fruit classification data 

If you have not already done so, you can use T4 to access the subset of fruit data from Google Open Images 

In [ ]:
#import Comet before all other imports 
from comet_ml import Experiment 
import t4

In [ ]:
# install the open fruit data to the directory of your choice (set via dest = "DIRECTORY_HERE")
t4.Package.install(
    "quilt/open_fruit", 
    registry="s3://quilt-example", 
    dest="./data") # set your own directory here 

### Create the Experiment object

In [17]:
# Define your Comet Experiment object (you'll need to pass in your API Key - here we set the API as an environment variable)
experiment = Experiment(project_name="comet-quilt-example")

COMET INFO: old comet version (1.0.45) detected. current: 1.0.46 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/ceceshao1/comet-quilt-example/a5ec4a7953a54bcd817e3b7aa9c11a48



### Confirm GPU usage and Imports 

In [48]:
# confirm Keras is running on GPU 
# If you are not using GPUs, skip this cell 
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [53]:
# If you are not using GPUs, skip this cell 
K.tensorflow_backend._get_available_gpus()

[]

In [50]:
from keras import applications
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import EarlyStopping

### Data Pre-processing and Parameters

In [22]:
# dimensions of our images
img_width, img_height = 150, 150

# set parameters
batch_size = 16
num_classes = 16
epochs = 50
activation = 'relu'
min_delta=0
patience=4
dropout=0.2
lr=0.0001

train_samples = 27593
validation_samples = 6889

In [23]:
params={'batch_size':batch_size,
        'num_classes':num_classes,
        'epochs':epochs,
        'min_delta':min_delta,
        'patience':patience,
        'learning_rate':lr,
        'dropout':dropout
}

experiment.log_parameters(params) #log these parameters as a dictionary to Comet. Adjust parameters in the cell above

In [46]:
from keras.applications.inception_v3 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2 #set the validation split 
)

test_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    './data/quilt/open_fruit/images_cropped',
    target_size=(150, 150),
    shuffle=True,
    seed=20,
    batch_size = batch_size,
    class_mode='categorical',
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    './data/quilt/open_fruit/images_cropped',
    target_size=(150, 150),
    seed=20,
    batch_size=batch_size,
    class_mode='categorical',
    subset = "validation"
)

Found 27593 images belonging to 16 classes.
Found 6889 images belonging to 16 classes.


In [ ]:
# Create a hash for your training data - https://www.comet.ml/docs/python-sdk/Experiment/#experimentlog_dataset_hash
Experiment.log_dataset_hash(train_generator)

### Building the base model

We are using the InceptionV3 model that has been pre-trained on ImageNet weights. Since we are going to adjust the classification task for our fruit classes and fine-tune the model, we're going to denote `include_top=False`.

In [8]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False,input_shape=(150,150))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- we have 16 classes for the fruits 
predictions = Dense(16, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
for i,layer in enumerate(base_model.layers):
    print(i,layer.name)

In [9]:
import pathlib
sample_size = len(list(pathlib.Path('./data/quilt/open_fruit/images_cropped').rglob('./*')))

In [ ]:
# train the model on the new data for a few epochs
model.fit_generator(
    train_generator,
    steps_per_epoch=sample_size // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size,
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience)]
)

At this point, the top layers are well trained and we can start fine-tuning convolutional layers from inception V3. We will freeze the bottom N layers and train the remaining top layers.

In [25]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [54]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [26]:
# freeze the first 249 layers and unfreeze the rest:
for layer in model.layers[:310]:
   layer.trainable = False
for layer in model.layers[310:]:
   layer.trainable = True

### Visualize training process in real-time

Using the `experiment.display()` method, you can view live visualizations of the model's training progress. Once you run the cell below with `model.fit_generator()`, training data will begin reporting and the visualizations in the **Chart** will render

In [ ]:
Experiment.display()

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate so we don't lose the value of the pre-trained model
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
    train_generator,1
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples// batch_size,
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience)]
)

### Save model weights

In [28]:
#save locally
model.save_weights('inceptionv3_tuned.h5') 

#save to Comet Asset Tab
# you can retrieve these weights later via the REST API 
experiment.log_asset(file_path='./inceptionv3_tuned.h5', file_name='inceptionv3_tuned.h5') 

{'web': 'https://www.comet.ml/api/asset/download?assetId=a6c75ebcfd344c06a4934b97641ea87e&experimentKey=a5ec4a7953a54bcd817e3b7aa9c11a48',
 'api': 'https://www.comet.ml/api/rest/v1/asset/get-asset?assetId=a6c75ebcfd344c06a4934b97641ea87e&experimentKey=a5ec4a7953a54bcd817e3b7aa9c11a48'}

In [29]:
# signal the end of the experiment 
# retrieve summary statistics around metrics, assets, etc...
experiment.end()

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:           acc: 0.7464916415897019
COMET INFO:         batch: 1720
COMET INFO:     epoch_end: 49
COMET INFO:          loss: 0.7562066181474528
COMET INFO:          size: 16
COMET INFO:          step: 86200
COMET INFO:   Other:
COMET INFO:     trainable_params: 23917360
COMET INFO:   Uploads:
COMET INFO:      assets: 1
COMET INFO:     figures: 0
COMET INFO:      images: 0
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading


### Accessing the full model code from Comet 

If you run this experiment from a git directory, you or your teammates can retrieve this notebook using the `Reproduce` button from the Comet experiment view. 

When you press the `Reproduce` button (see screenshot below), you will see a prompt to download the notebook. The notebook will contain the cells in the order they were excecuted. 

![](../images/comet-reproduce-notebook.png)


**Retrieving a script:**
If you train a model with a script instead, you will be able to see bash commands to retreive the code (including untracked changes).
![](../images/comet reproduce script.png)
